# 数据预处理过程

## author 张念

In [25]:
import pandas as pd
import seaborn as sns
import matplotlib as plt
import numpy as np
import missingno as msno
%matplotlib inline

pd.options.display.float_format = '{:.0f}'.format
pd.set_option('display.max_columns',100)
df=pd.read_csv('20180105.csv',encoding='gbk')
# 列出前五行
df.head()

,ID,SRC_ADD,CALLING_NUM,VDN,DIALED_NUM,DURATION,CALLING_DATE,CALLING_TIME,COND_CODE,IN_TRK_CODE,FRL,IN_CRT_ID,CODE_USED,MAKER,CIRCUIT_TYPE,CALLERLOC,CALL_NUM,CALL_TYPE,CHARGE_RULE_ID,COST,CALL_DATE,CRE_DATE,MAKER_NAME,ADDRESS,MOBILE_MAKER_NAME,PROVINCE_NAME
0,1753245405,80.14.28.10,4886,76318,9885029#,78,10518,0,7,nan,1,nan,4722,8,4,NaN,3,2,0,0,2018/1/5,2018/1/5 0:01:45,IP Trunk,SIP中继(思科）,NaN,NaN
1,1753245406,80.14.28.10,4894,76308,9885009#,78,10518,0,7,nan,1,nan,4706,8,4,NaN,3,2,0,0,2018/1/5,2018/1/5 0:01:45,IP Trunk,SIP中继(思科）,NaN,NaN
2,1753245407,80.14.28.10,4823,76322,9885037#,48,10518,0,7,nan,1,nan,4710,8,4,NaN,3,2,0,0,2018/1/5,2018/1/5 0:01:45,IP Trunk,SIP中继(思科）,NaN,NaN
3,1753245408,80.14.28.10,4890,76314,9885023#,54,10518,0,7,nan,1,nan,4706,8,4,NaN,3,2,0,0,2018/1/5,2018/1/5 0:01:45,IP Trunk,SIP中继(思科）,NaN,NaN
4,1753245409,80.14.28.10,4880,76319,9885030#,42,10518,0,7,nan,1,nan,4714,8,4,NaN,3,2,0,0,2018/1/5,2018/1/5 0:01:45,IP Trunk,SIP中继(思科）,NaN,NaN


In [26]:
# filter dailed out & 95588 & area_code + 95588
df = df[(df['CALL_TYPE'] == 1) & \
        (df['CALLING_NUM']/1000000 >= 1) & ((df['CALLING_NUM']%100000 != 95588) | (df['CALLING_NUM']/100000000 >= 1))]

# filter test numbers
df = df[df['CALLING_NUM'].isin([18210261957, 18210261795, 17343195721, \
                               17718332497, 13263105230, 13263103918]) == False]

# filter IN_TRK_CODE
df = df[df['IN_TRK_CODE'].isin([4860,4881,4815,4816,4834,4873,4878,4899,4820,4862,\
                                4849,4850,4828,4827,4858,4859,4840,4829,4848,4830,\
                                4831,4832,4833,4847,4854,4705,4706,4707,4708,4709,\
                                4710,4711,4712,4713,4714,4855,4715,4716,4717,4718,\
                                4719,4720,4721,4722,4723,4724,4856,4725,4726,4727,\
                                4728,4729,4730,4731,4732,4733,4857,4700,4701,4702,\
                                4703,4704,4872,4804,4894,4801,4898,4821,4805,4897,\
                                4806,4802,4892,4893,4890,4861,4870,4826,4885,4886,\
                                4810,4811,4852,4853,4880,4879,4807,4823,4876,4877,\
                                4808,4803,4812])]

df.shape

(499622, 26)

# 数据预处理

In [27]:
import numpy as np

In [28]:
# df = df[:10000]
#出现次数
def count(x):
    return len(np.unique(x))

#MOST_DURATION
def most_appearance(x):
    return x.value_counts().nlargest(1).index[0]

#时间转成分钟
def get_minute(dt):
    dt = str(dt)
    dt_len = len(dt)
    if dt_len < 3:
        dt_min = int(dt)
        dt_hour = 0
    else:
        dt_min = int(dt[-2:])
        dt_hour = int(dt[:-2])
    return dt_hour * 60 + dt_min


def get_interval(x, y):
    return abs(get_minute(y) - get_minute(x))

#获得通话间隔列表
def interval(x):
    x_1 = x[:-1]
    x_2 = x[1:]
    intervals = map(get_interval, x_1, x_2)
    return intervals

#获取间隔中位数
def interval_median(x):
    if len(x) > 1:
        intervals = np.array(interval(x))
        return  np.median(np.array(intervals))
    return 60*24

#获取间隔标准差
def interval_std(x):
    if len(x) > 1:
        intervals = np.array(interval(x))
        return  np.std(np.array(intervals))
    return np.std(np.array([60*24]))


result = pd.pivot_table(df,index=["CALLING_NUM"],values=["DURATION","IN_TRK_CODE","CALLING_TIME"],
               aggfunc={"DURATION":[np.median,np.sum,np.std,most_appearance,count,len],
                        "IN_TRK_CODE":[count],"CALLING_TIME":[interval_median,interval_std]},fill_value=0)

result.to_csv('result.csv',index=True)

In [24]:
result.isnull().sum()

CALLING_TIME  interval_median    0
              interval_std       0
DURATION      count              0
              len                0
              median             0
              most_appearance    0
              std                0
              sum                0
IN_TRK_CODE   count              0
dtype: int64

In [16]:
print type(result)
for i in result:
    print i

<class 'pandas.core.frame.DataFrame'>
('CALLING_TIME', 'interval_median')
('CALLING_TIME', 'interval_std')
('DURATION', 'count')
('DURATION', 'len')
('DURATION', 'median')
('DURATION', 'most_appearance')
('DURATION', 'std')
('DURATION', 'sum')
('IN_TRK_CODE', 'count')
